# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [30]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [31]:
#Your code here; create the connection and cursor
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [33]:
#Your code here; use a subquery. No join will be necessary.

c.execute('''select customerNumber, contactLastName, contactFirstName 
                    from customers where customerNumber in
                    (select customerNumber from orders where orderDate = '2003-01-31')''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [34]:
#Your code here

c.execute('''select productName, count(distinct orderNumber), sum(quantityOrdered) 
                    from products join orderdetails 
                    using(productCode) group by 1 order by 3 DESC''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()
# c.execute('''select productName, quantityOrdered from products
#                     where quantityOrdered in (select quantityOrdered from orderdetails) group by 1''')

,productName,count(distinct orderNumber),sum(quantityOrdered)
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [37]:
#Your code here
c.execute('''select productName, count(distinct customerNumber) from products
                    join orderdetails using(productCode)
                    join orders using(orderNumber) group by 1 order by 2 DESC''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,count(distinct customerNumber)
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [69]:
#Your code here

#need access to 3 tables: employees, office, and 

c.execute('''select firstname, 
                    lastname, 
                    employeeNumber, 
                    officeCode, 
                    city 
                    from employees 
                    join offices using(officeCode) 
                    join customers on salesrepemployeeNumber = employeeNumber
                    join orders using(customerNumber)
                    where employeeNumber in 
                    (select employeeNumber 
                    from employees
                    join customers 
                    on salesrepemployeeNumber = employeeNumber 
                    join orders using(customernumber)
                    join orderdetails using(orderNumber) 
                    group by 1 
                    having count(distinct orderNumber) <20)''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

# where employeeNumber in (select employeeNumber from orderNumber < 20)

,firstName,lastName,employeeNumber,officeCode,city
0,Leslie,Thompson,1166,1,San Francisco
1,Julie,Firrelli,1188,2,Boston
2,Steve,Patterson,1216,2,Boston
3,Foon Yue,Tseng,1286,3,NYC
4,Andy,Fixter,1611,6,Sydney
5,Peter,Marsh,1612,6,Sydney
6,Mami,Nishi,1621,5,Tokyo
7,Martin,Gerard,1702,4,Paris


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [67]:
#Your code here
c.execute('''select firstname, lastname, salesrepemployeeNumber
                    from employees join customers on salesrepemployeeNumber = employeeNumber
                    where salesrepemployeeNumber in (select salesrepemployeeNumber from customers 
                    group by 1 having avg(creditlimit) > 15000)''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,firstName,lastName,salesRepEmployeeNumber
0,Gerard,Hernandez,1370
1,Leslie,Thompson,1166
2,Andy,Fixter,1611
3,Gerard,Hernandez,1370
4,Barry,Jones,1504
5,Leslie,Jennings,1165
6,Barry,Jones,1504
7,Leslie,Jennings,1165
8,George,Vanauf,1323
9,Gerard,Hernandez,1370


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!